In [89]:
import pandas as pd
import os

In [91]:
APPROACH = "spinfer"

In [92]:
df = pd.read_csv(f"computed_{APPROACH}.csv")
df["id"] = [x for x in range(len(df))]
df.head(3)

,root,cocci_filename,c_filename,pred,is_cocci_valid,is_applied,diff_truth,diff_pred,diff_5,precision,recall,cocci_fullpath,rank_score,cocci,id
0,./EXP0-7,sp_out.final.spinfer.cocci,1533533124_2018-08-06_4fd786e6c3d6_qgroup_btrf...,out,1,True,"- if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_i...",NaN,"--- initial\n+++ final\n@@ -1,10 +1,10 @@\n vo...",0.0,0.0,./EXP0-7/sp_out.final.spinfer.cocci,0.333333,"@@\nexpression E0, E1;\n@@\n- E0->objectid = E...",0
1,./EXP0-7,sp_out.final.spinfer.cocci,1533533124_2018-08-06_4fd786e6c3d6_super_btrfs...,out,1,True,- buf->f_fsid.val[0] ^= BTRFS_I(d_inode(dentr...,NaN,"--- initial\n+++ final\n@@ -65,9 +65,9 @@\n ...",0.0,0.0,./EXP0-7/sp_out.final.spinfer.cocci,0.333333,"@@\nexpression E0, E1;\n@@\n- E0->objectid = E...",1
2,./EXP0-7,sp_out.final.spinfer.cocci,1533533124_2018-08-06_4fd786e6c3d6_ctree_add_r...,out,1,True,- if (root->objectid == BTRFS_EXTENT_TREE...,NaN,"--- initial\n+++ final\n@@ -5,11 +5,11 @@\n ...",0.0,0.0,./EXP0-7/sp_out.final.spinfer.cocci,0.333333,"@@\nexpression E0, E1;\n@@\n- E0->objectid = E...",2


# Analyze valid semantic patches that fail to transform unseen C codes (0 Precision and 0 Recall)

In [93]:
mask1 = df["precision"] == 0
mask3 = df["recall"] == 0
mask2 = df["is_cocci_valid"] == 1
valid_sp_fail_transform = df[(mask1) & (mask2) & (mask3)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_fail = len(valid_sp_fail_transform)

In [97]:
valid_sp_fail_transform[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [98]:
count = 1
for idx, row in valid_sp_fail_transform.iterrows():
    print(count)
    count += 1
    # print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    debug_filepath = os.path.join(row["root"], row["c_filename"] + f".out.{APPROACH}.res.txt")
    with open(debug_filepath, "r") as f:
        debug = f.read()


    print(debug)
    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
init_defs_builtins: /usr/bin/../lib/coccinelle/standard.h
-----------------------------------------------------------------------
processing semantic patch file: ./EXP0-7/sp_out.final.spinfer.cocci
with isos from: /usr/bin/../lib/coccinelle/standard.iso
-----------------------------------------------------------------------
@@
expression E0, E1;
@@
- E0->objectid = E1; 
+ E0->root_key.objectid = E1; 
// Infered from: (./EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_disk-io___setup_root.{c.sanitized.c,res.c.sanitized.res.c}: __setup_root)
// False positives: (./EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_export_btrfs_encode_fh.res.c.sanitized.res.c: btrfs_encode_fh)
// Recall: 0.14, Precision: 0.50, Matching recall: 0.33

// ---------------------------------------------
@@
expression E0, E1;
@@
- E0 = BTRFS_I(E1)->root->objectid; 
+ E0 = BTRFS_I(E1)->root->root_key.objectid; 
// Infered from: (./EXP0-7/1533533124_2018-08-06_4fd786e6c3d6_export_btrfs_encode_fh.{c.sanitized.c,res.c.sanitized.

In [99]:
count = 1
for idx, row in valid_sp_fail_transform.iterrows():
    print(count)
    count += 1
    print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
--- initial
+++ final
@@ -1,10 +1,10 @@
 void btrfs_qgroup_free_meta_all_pertrans(struct btrfs_root *root)
 {
   struct btrfs_fs_info *fs_info = root->fs_info;
-  if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_info->flags) || !is_fstree(root->objectid))
+  if (!test_bit(BTRFS_FS_QUOTA_ENABLED, &fs_info->flags) || !is_fstree(root->root_key.objectid))
     return;
 
   trace_qgroup_meta_free_all_pertrans(root);
 
-  btrfs_qgroup_free_refroot(fs_info, root->objectid, (u64)-1, BTRFS_QGROUP_RSV_META_PERTRANS);
+  btrfs_qgroup_free_refroot(fs_info, root->root_key.objectid, (u64)-1, BTRFS_QGROUP_RSV_META_PERTRANS);
 }
#############################################################
#############################################################
#############################################################
2
--- initial
+++ final
@@ -65,9 +65,9 @@
   buf->f_namelen = BTRFS_NAME_LEN;
 
   buf->f_fsid.val[0] = be32_to_cpu(fsid[0]) ^ be32_to_cpu(fsid[2]);
   buf->f_fsid.val[1] = be32_to_cpu(fsid[1]) ^ be

# Analyze valid semantic patches that fail to transform unseen C codes PARTIALLY (Precision > 0 and Recall > 0, but both still less than 1) 

In [100]:
mask1 = df["precision"] != 0
mask2 = df["precision"] != 1
mask3 = df["recall"] != 0
mask4 = df["recall"] != 1
mask5 = df["is_cocci_valid"] == 1
partial_sp = df[(mask1) & (mask2) & (mask3) & (mask4) & (mask5)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_partial = len(partial_sp)

In [101]:
partial_sp[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [104]:
count = 1
for idx, row in partial_sp.iterrows():
    print(count)
    count += 1
    # print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    debug_filepath = os.path.join(row["root"], row["c_filename"] + f".out.{APPROACH}.res.txt")
    with open(debug_filepath, "r") as f:
        debug = f.read()


    print(debug)
    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
init_defs_builtins: /usr/bin/../lib/coccinelle/standard.h
-----------------------------------------------------------------------
processing semantic patch file: ./random_ether_addr-84/sp_out.final.spinfer.cocci
with isos from: /usr/bin/../lib/coccinelle/standard.iso
-----------------------------------------------------------------------
@@
identifier I1;
expression E0;
@@
- random_ether_addr(E0->I1); 
+ eth_random_addr(E0->I1); 
// Infered from: (./random_ether_addr-84/1529689860_2018-06-22_6c1f0a1ffb7c_rmnet_vnd_rmnet_vnd_setup.{c.sanitized.c,res.c.sanitized.res.c}: rmnet_vnd_setup), (./random_ether_addr-84/1529689860_2018-06-22_6c1f0a1ffb7c_qlcnic_sriov_common_qlcnic_sriov_init.{c.sanitized.c,res.c.sanitized.res.c}: qlcnic_sriov_init), (./random_ether_addr-84/1529689860_2018-06-22_6c1f0a1ffb7c_gemini_gemini_ethernet_port_probe.{c.sanitized.c,res.c.sanitized.res.c}: gemini_ethernet_port_probe)
// Recall: 1.00, Precision: 1.00, Matching recall: 1.00

// -----------------------------

In [ ]:
count = 1
for idx, row in partial_sp.iterrows():
    print(count)
    count += 1
    print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


1
--- initial
+++ final
@@ -6,11 +6,11 @@
   nic_data->vf = kcalloc(efx->vf_count, sizeof(struct ef10_vf), GFP_KERNEL);
   if (!nic_data->vf)
     return -ENOMEM;
   for (i = 0; i < efx->vf_count; i++)
     {
-      random_ether_addr(nic_data->vf[i].mac);
+      eth_random_addr(nic_data->vf[i].mac);
       nic_data->vf[i].efx = NULL;
       nic_data->vf[i].vlan = EFX_EF10_NO_VLAN;
       rc = efx_ef10_sriov_assign_vf_vport(efx, i);
       if (rc)
         goto fail;
#############################################################
#############################################################
#############################################################
2
--- initial
+++ final
@@ -14,11 +14,11 @@
       return ret;
     }
   mgabo->bo.bdev = &mdev->ttm.bdev;
   mgag200_ttm_placement(mgabo, TTM_PL_FLAG_VRAM | TTM_PL_FLAG_SYSTEM);
   acc_size = ttm_bo_dma_acc_size(&mdev->ttm.bdev, size, sizeof(struct mgag200_bo));
-  ret = ttm_bo_init(&mdev->ttm.bdev, &mgabo->bo, size, ttm_bo_type_device, &mg

# Analyze invalid semantic patches 

In [105]:
mask1 = df["precision"] == 0
mask3 = df["recall"] == 0
mask5 = df["is_cocci_valid"] != 1
invalid_sp = df[(mask1) & (mask3) & (mask5)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_invalid = len(invalid_sp)

In [106]:
invalid_sp[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [107]:
invalid_sp.root.value_counts()

root
./tcf_block_get-61    27
./snd_soc             20
Name: count, dtype: int64

# Analyze valid semantic patches

In [108]:
mask1 = df["precision"] == 1
mask3 = df["recall"] == 1
mask5 = df["is_cocci_valid"] == 1
valid_sp = df[(mask1) & (mask3) & (mask5)]
# temp_df = temp_df.sample(n=50, random_state=123)
# temp_df.sort_values(["root", "c_filename"], inplace=True)
count_correct = len(valid_sp)

In [109]:
(count_correct+count_fail+count_invalid+count_partial == len(df))

True

In [61]:
count_correct

61

In [62]:
count_fail

32

In [63]:
count_invalid

83

In [64]:
count_partial

1

In [65]:
72+32+65+5

174

In [66]:
temp = pd.concat([invalid_sp, partial_sp, valid_sp, valid_sp_fail_transform])

In [67]:
temp = df[~df.id.isin(temp.id)]

In [68]:
temp[["root", "c_filename"]].to_csv("temp.csv", index=False)

In [69]:
count = 1
for idx, row in temp.iterrows():
    print(count)
    count += 1
    # print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    debug_filepath = os.path.join(row["root"], row["c_filename"] + f".out.{APPROACH}.res.txt")
    with open(debug_filepath, "r") as f:
        debug = f.read()


    print(debug)
    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    


In [70]:
count = 1
for idx, row in temp.iterrows():
    print(count)
    count += 1
    print(row["diff_5"])
    # cocci_filepath = os.path.join(row["root"], row["cocci_filename"])
    # print(cocci_filepath)

    print("#############################################################")
    print("#############################################################")
    print("#############################################################")
    
